In [2]:


import os
import re
import json
import pandas as pd
from sqlalchemy import create_engine, text


# 1. Connects

USER     = os.getenv("MYSQL_USER", "analyst")
PASSWORD = os.getenv("MYSQL_PASSWORD", "password")
HOST     = os.getenv("MYSQL_HOST", "host.docker.internal")
PORT     = os.getenv("MYSQL_PORT", "3306")
DB       = os.getenv("MYSQL_DATABASE", "analytics_db")

URI = f"mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}"
engine = create_engine(URI)


OUTPUT_DIR = "reports"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# 2. SQL-queries 

queries = {
    "task1_top5_campaigns_ctr_30d": """
        SELECT
          cam.name,
          COUNT(fe.event_id)                     AS total_impressions,
          COALESCE(SUM(fe.was_clicked), 0)        AS total_clicks,
          ROUND(
            SUM(fe.was_clicked) * 1.0
            / NULLIF(COUNT(fe.event_id), 0)
          , 5)                                      AS ctr
        FROM dim_campaign AS cam
        LEFT JOIN (
            SELECT *
            FROM fact_event
            WHERE DATE(timestamp) BETWEEN '2024-12-01' AND '2024-12-30'
        ) AS fe
          ON cam.campaign_id = fe.campaign_id
        GROUP BY cam.name
        ORDER BY ctr DESC
        LIMIT 5
    """,

    "task2_top10_advertisers_spend_engagement": """
        SELECT
          adv.name,
          COUNT(fe.event_id)                     AS total_impressions,
          COALESCE(SUM(fe.was_clicked), 0)        AS total_clicks,
          ROUND(
            SUM(fe.was_clicked) * 1.0
            / NULLIF(COUNT(fe.event_id), 0)
          , 5)                                      AS ctr,
          COALESCE(SUM(fe.ad_cost), 0)           AS total_spending
        FROM dim_campaign AS cam
        LEFT JOIN (
            SELECT *
            FROM fact_event
            WHERE DATE(timestamp) BETWEEN '2024-12-01' AND '2024-12-30'
        ) AS fe
          ON cam.campaign_id = fe.campaign_id
        LEFT JOIN dim_advertiser AS adv
          ON cam.advertiser_id = adv.advertiser_id
        GROUP BY adv.name
        ORDER BY total_spending DESC
        LIMIT 10
    """,

    "task3_cpc_cpm_per_campaign": """
        SELECT
          cam.name,
          COUNT(fe.event_id)                     AS total_impressions,
          COALESCE(SUM(fe.was_clicked), 0)        AS total_clicks,
          ROUND(
            SUM(fe.ad_cost) * 1.0
            / NULLIF(SUM(fe.was_clicked), 0)
          , 2)                                      AS cpc,
          ROUND(
            SUM(fe.ad_cost) * 1.0
            / NULLIF(COUNT(fe.event_id), 0) * 1000
          , 2)                                      AS cpm
        FROM fact_event AS fe
        LEFT JOIN dim_campaign AS cam
          ON fe.campaign_id = cam.campaign_id
        GROUP BY cam.name
    """,

    "task4_revenue_by_country": """
        SELECT
          tar.country,
          SUM(fe.ad_revenue)                     AS total_revenue
        FROM dim_targeting AS tar
        LEFT JOIN fact_event AS fe
          ON tar.campaign_id = fe.campaign_id
        GROUP BY tar.country
        ORDER BY total_revenue DESC
    """,

    "task5_user_engagement_revenue_top20": """
        SELECT
          user.user_id,
          user.location,
          user.device,
          COALESCE(SUM(fe.ad_revenue), 0)       AS total_revenue
        FROM dim_user AS user
        LEFT JOIN fact_event AS fe
          ON user.user_id = fe.user_id
        GROUP BY user.user_id, user.location, user.device
        ORDER BY total_revenue DESC
        LIMIT 20
    """,

    "task6_budget_consumption_over80": """
        SELECT
          cam.name,
          cam.budget,
          fe.total_cost,
          ROUND(fe.total_cost / cam.budget * 100, 1) AS pct_spent
        FROM dim_campaign AS cam
        LEFT JOIN (
            SELECT
              campaign_id,
              COALESCE(SUM(ad_cost), 0)         AS total_cost
            FROM fact_event
            GROUP BY campaign_id
        ) AS fe
          ON cam.campaign_id = fe.campaign_id
        WHERE fe.total_cost / cam.budget >= 0.8
        ORDER BY pct_spent DESC
        LIMIT 10
    """,

    "task7_ctr_by_device": """
        SELECT
          user.device,
          ROUND(SUM(fe.was_clicked) * 1.0 / COUNT(fe.event_id), 4) AS ctr
        FROM dim_user AS user
        LEFT JOIN fact_event AS fe
          ON user.user_id = fe.user_id
        GROUP BY user.device
        ORDER BY ctr DESC
    """
}

# 3. SQL-queris run and saving results

full_report = {}

for report_name, sql in queries.items():
    safe_filename = re.sub(r'[^0-9a-zA-Z_]+', '_', report_name).lower()
    csv_path = os.path.join(OUTPUT_DIR, f"{safe_filename}.csv")

    print(f"Running report `{report_name}` → {csv_path}")
    try:
        df = pd.read_sql_query(text(sql), engine)
        df.to_csv(csv_path, index=False)
        print(f"  • CSV saved to {csv_path}")

        full_report[report_name] = df.to_dict(orient="records")
    except Exception as e:
        print(f"  ⚠️ Error in report `{report_name}`: {e}")


json_path = os.path.join(OUTPUT_DIR, "full_report.json")
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(full_report, f, ensure_ascii=False, indent=2)
print(f"• Full JSON report saved to {json_path}")


# 4. Engine close

engine.dispose()
print("Done.")


Running report `task1_top5_campaigns_ctr_30d` → reports/task1_top5_campaigns_ctr_30d.csv
  • CSV saved to reports/task1_top5_campaigns_ctr_30d.csv
Running report `task2_top10_advertisers_spend_engagement` → reports/task2_top10_advertisers_spend_engagement.csv
  • CSV saved to reports/task2_top10_advertisers_spend_engagement.csv
Running report `task3_cpc_cpm_per_campaign` → reports/task3_cpc_cpm_per_campaign.csv
  • CSV saved to reports/task3_cpc_cpm_per_campaign.csv
Running report `task4_revenue_by_country` → reports/task4_revenue_by_country.csv
  • CSV saved to reports/task4_revenue_by_country.csv
Running report `task5_user_engagement_revenue_top20` → reports/task5_user_engagement_revenue_top20.csv
  • CSV saved to reports/task5_user_engagement_revenue_top20.csv
Running report `task6_budget_consumption_over80` → reports/task6_budget_consumption_over80.csv
  • CSV saved to reports/task6_budget_consumption_over80.csv
Running report `task7_ctr_by_device` → reports/task7_ctr_by_device.csv